### PGE Creation
This notebook is the driver by which the *hello_world_notebook* will be processed in order to be available as an SDS PGE. The notebook *hello_world_submit_job* is then used to submit it to the SDS for execution.


#### 1. Set-up
To start, we; enter the directory containing the notebook to be executed (this step assumes the repository has already been cloned locally), ensure that we are in the proper branch, then execute notebook-pge-wrapper to generate the files necessary to build and run the *papermill*'ed notebook as a PGE.

Make note of the branch being used (e.g. *develop* in this example), it will become part of the job specification name and used when submitting the job in the next notebook.

In [1]:
%%bash
cd ~/hello_world
git checkout develop
notebook-pge-wrapper specs all

M	hello_world_pge_create.ipynb
Your branch is up to date with 'origin/develop'.
.ipynb_checkpoints is not a notebook, skipping...
inspecting notebook: hello_world_notebook.ipynb
generated docker/hysds-io.json.hello_world_notebook
generated docker/job-spec.json.hello_world_notebook


Already on 'develop'
2021-02-05 22:48:37 [INFO] Input Notebook:  notebook_pges/hello_world_notebook.ipynb
2021-02-05 22:48:37 [INFO] Input Notebook:  notebook_pges/hello_world_notebook.ipynb
2021-02-05 22:48:37 [INFO] Input Notebook:  notebook_pges/hello_world_notebook.ipynb


### Manual edits.

##### 1. Edit docker/job-spec.json.hello_world_notebook
A manual edit to the job-spec file is necessary. In the line:

>`    "command": "notebook-pge-wrapper execute /home/ops/hello_world/notebook_pges/hello_world_notebook.ipynb",`
 
Change *'ops'* to *'jovyan'*:

>`    "command": "notebook-pge-wrapper execute /home/jovyan/hello_world/notebook_pges/hello_world_notebook.ipynb",`
 
##### 2. Update the docker/Dockerfile
The contents of the Dockerfile should be as follows:

```
FROM cae-artifactory.jpl.nasa.gov:16001/gov/nasa/jpl/nisar/sds/pcm/nisar_ade:r1.3

ENV HOME=/home/jovyan \
    CONDA_DIR=/opt/conda

USER root
RUN $CONDA_DIR/bin/conda install -y nb_conda_kernels
RUN $CONDA_DIR/bin/pip install 'ipykernel<5.0.0'

RUN cd $HOME \
    && git clone https://github.com/hysds/notebook_pge_wrapper.git \
    && cd notebook_pge_wrapper \
    && $CONDA_DIR/bin/pip install -e .

USER jovyan

# copy your repo into the docker container
################################################
COPY . $HOME/hello_world
################################################

WORKDIR $HOME

CMD ["/bin/bash", "--login"]
```

#### 2. Commit changes
The notebook as a PGE is obtained by the system via a git checkout, in this case from github, so we need to make sure the remote repository is up to date. There may not be anything to commit, but we need to ensure consistency - there is no harm in trying to commit in the absence of changes.

In [2]:
%%bash
cd ~/hello_world
git add -A
git commit -m"Results of notebook-pge-wrapper spec generation."
git push

[develop 9f85148] Results of notebook-pge-wrapper spec generation.
 1 file changed, 38 insertions(+), 8 deletions(-)


To https://github.com/jimmie/hello_world
   1b74a94..9f85148  develop -> develop


#### 3. Initialize the api that will be used to specify the PGE job to HySDS

In [5]:
import os
from pathlib import Path
import otello
if not os.path.exists(f"{Path.home()}/.config/otello/config.yml"):
    otello.client.initialize()

/home/jovyan/.config/otello/config.yml not found



HySDS host:  https://100.64.122.62
HySDS cluster authenticated (y/n):  n



auth: false
host: https://100.64.122.62



#### 4. Specify the repository and branch to the api
This creates a handle to the job specification.

In [6]:
repository = "https://github.com/jimmie/hello_world.git"
branch = "develop"

ci = otello.CI(repo=repository, branch=branch)

In [7]:
ci.check_job_exists()

True

#### 5. Ensure the most current version of the job is registered
If the job exists previously, unregister it. Regardless, register the job to ensure the most current version. 

In [ ]:
try:
    if (ci.check_job_exists()):
        print("Job exists, unregistering")
        ci.unregister()
except Exception as e:
    print("Exception : {}, trying to register anyway".format(e))

ci.register()

#### 6. Build the PGE

In [ ]:
ci.submit_build()

#### 7. Wait for the build to complete, then sample the result.

In [ ]:
import time
move_on = False
while not move_on:
    try:
        ci.get_build_status()
        move_on = True
    except Exception as e:
        print("Waiting for build to start")
        time.sleep(10)
    
build_stat = ci.get_build_status()
print("Build in progress.", end="")

while (build_stat['building']):
    time.sleep(10)
    print(".", end="", flush=True)
    build_stat = ci.get_build_status()

if (build_stat['result'] == 'FAILURE'):
    print("\nBuild Failed.")
else:
    print("\nBuild completed successfully")

#### 8. Get the final details on the build completion.

In [ ]:
build_stat = ci.get_build_status()
print(build_stat)
if (build_stat['result'] == 'FAILURE'):
    print("Build Faild.")
else:
    print("Build completed successfully")